In [2]:
# All imports

import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import scipy.io as sio
import pandas as pd
import numpy as np

from skimage.feature import hessian_matrix_det
from skimage.feature import corner_harris, corner_subpix, corner_peaks


2022-05-29 20:55:48.060011: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-29 20:55:48.060042: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
# Load the MPII dataset
#

mat = sio.loadmat('mpii_human_pose_v1_u12_2/mpii_human_pose_v1_u12_1.mat', struct_as_record=False)

In [4]:
rel = mat['RELEASE']

obj_rel = rel[0,0]

annolist = obj_rel.__dict__['annolist']
img_tra = obj_rel.__dict__['img_train']
act = obj_rel.__dict__['act']

n = annolist.shape[1]

[[<scipy.io.matlab._mio5_params.mat_struct object at 0x7f19586d6220>]] <class 'numpy.ndarray'> (1, 1)


In [9]:
# All the columns of the data 

data_arr = ['NAME','r ankle_X','r ankle_Y', 'r knee_X','r knee_Y', 'r hip_X','r hip_Y', 'l hip_X','l hip_Y', 'l knee_X','l knee_Y', 'l ankle_X','l ankle_Y','pelvis_X','pelvis_Y','thorax_X','thorax_Y','upper neck_X','upper neck_Y', 'head top_X','head top_Y', 'r wrist_X','r wrist_Y','r elbow_X','r elbow_Y', 'r shoulder_X','r shoulder_Y','l shoulder_X','l shoulder_Y','l elbow_X','l elbow_Y', 'l wrist_X','l wrist_Y','Scale','Activity','Category']

data = pd.DataFrame(columns=data_arr)

['NAME', 'r ankle_X', 'r ankle_Y', 'r knee_X', 'r knee_Y', 'r hip_X', 'r hip_Y', 'l hip_X', 'l hip_Y', 'l knee_X', 'l knee_Y', 'l ankle_X', 'l ankle_Y', 'pelvis_X', 'pelvis_Y', 'thorax_X', 'thorax_Y', 'upper neck_X', 'upper neck_Y', 'head top_X', 'head top_Y', 'r wrist_X', 'r wrist_Y', 'r elbow_X', 'r elbow_Y', 'r shoulder_X', 'r shoulder_Y', 'l shoulder_X', 'l shoulder_Y', 'l elbow_X', 'l elbow_Y', 'l wrist_X', 'l wrist_Y', 'Scale', 'Activity', 'Category']


In [37]:
# This dataset is in .mat format.
# 
# Convert the data into dataframe

for ix in range(0,annolist.shape[1]):
    if img_tra[0,ix] == 0:
        continue
    temp_arr = []
    obj_list = annolist[0,ix]
    obj_act = act[ix,0]
    
    rect =obj_list.__dict__['annorect']
    img_d = obj_list.__dict__['image']
    if rect.shape[0] ==0:
        continue
        
    obj_rect = rect[0,0]
    obj_img = img_d[0,0]
    
    
    if 'annopoints' not in obj_rect._fieldnames:
        continue
    
    name_d = obj_img.__dict__['name']
    name = name_d[0]
    temp_arr.append(name)
    annopoints = obj_rect.__dict__['annopoints']
    if annopoints.shape[0]==0:
        continue
    obj_points = annopoints[0,0]
    points = obj_points.__dict__['point']
    cnt = 0
    px = 0
    
    for n in range(0,32):
        temp_arr.append(-1)
    
    
    for px in range(0,points.shape[1]):
        po = points[0,px]
        po_id = po.__dict__['id']
        
        po_x = po.__dict__['x']
        po_y = po.__dict__['y']
        po_isvisible = po.__dict__['is_visible']
        ind = 2*po_id[0][0]+1
        temp_arr[ind] = po_x[0][0]
        temp_arr[ind+1] = po_y[0][0]
        
        
    scale = obj_rect.__dict__['scale']
    temp_arr.append(scale[0][0])
    
    activity = act[ix,0]
    
    a_n = activity.act_name
    c_n = activity.cat_name
    
    if a_n.shape[0]==0:
        temp_arr.append(a_n)
    else:
        temp_arr.append(activity.act_name[0])
    if c_n.shape[0]==0:
        temp_arr.append(c_n)
    else:
        temp_arr.append(activity.cat_name[0])
    
    temp_data_f = pd.DataFrame([temp_arr],columns=data_arr)
    
    data = pd.concat([data,temp_data_f])

IndexError: index 0 is out of bounds for axis 0 with size 0

In [12]:
data.tail(10)

,NAME,r ankle_X,r ankle_Y,r knee_X,r knee_Y,r hip_X,r hip_Y,l hip_X,l hip_Y,l knee_X,...,r shoulder_Y,l shoulder_X,l shoulder_Y,l elbow_X,l elbow_Y,l wrist_X,l wrist_Y,Scale,Activity,Category
0,049123457.jpg,-1,-1,438,373,-3,391,273,336,392,...,-19,-71,-74,-7,-79,335,-56,3.855718,driving automobile or light truck,transportation
0,057450433.jpg,-1,-1,433,376,-26,410,266,335,406,...,-33,-27,-70,284,260,329,276,3.711008,driving automobile or light truck,transportation
0,028705661.jpg,-1,-1,435,366,-3,391,273,336,399,...,-34,-57,-62,-31,277,295,271,3.806403,driving automobile or light truck,transportation
0,066294927.jpg,-1,-1,448,371,285,401,292,338,405,...,-13,-42,-74,-7,276,299,299,3.908759,driving automobile or light truck,transportation
0,050240222.jpg,607,689,599,542,605,416,691,410,704,...,-24,663,-28,623,289,555,333,3.480000,pushing car,transportation
0,033474347.jpg,876,461,815,380,779,296,742,303,706,...,-42,663,-33,674,286,631,260,2.003677,pushing car,transportation
0,082650067.jpg,1038,656,950,504,903,328,831,337,811,...,-52,729,-36,704,332,675,269,3.235820,pushing car,transportation
0,072772110.jpg,562,617,558,539,553,452,511,458,529,...,377,490,378,470,411,461,414,1.494952,pushing car,transportation
0,039361034.jpg,496,377,466,322,510,278,540,280,558,...,-3,466,-5,436,293,410,325,1.459468,pushing car,transportation
0,084761779.jpg,362,350,347,297,350,-32,317,-30,297,...,-88,288,-79,279,-57,268,-62,1.189877,pushing car,transportation


In [17]:
# Clean the data 
filter_data = data[data[data_arr] != 0].dropna(axis=0)
filter_data.tail(10)

/home/abhi/.local/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = libops.scalar_compare(x.ravel(), y, op)


,NAME,r ankle_X,r ankle_Y,r knee_X,r knee_Y,r hip_X,r hip_Y,l hip_X,l hip_Y,l knee_X,...,r shoulder_Y,l shoulder_X,l shoulder_Y,l elbow_X,l elbow_Y,l wrist_X,l wrist_Y,Scale,Activity,Category
0,049123457.jpg,-1,-1,438,373,-3,391,273,336,392,...,-19,-71,-74,-7,-79,335,-56,3.855718,driving automobile or light truck,transportation
0,057450433.jpg,-1,-1,433,376,-26,410,266,335,406,...,-33,-27,-70,284,260,329,276,3.711008,driving automobile or light truck,transportation
0,028705661.jpg,-1,-1,435,366,-3,391,273,336,399,...,-34,-57,-62,-31,277,295,271,3.806403,driving automobile or light truck,transportation
0,066294927.jpg,-1,-1,448,371,285,401,292,338,405,...,-13,-42,-74,-7,276,299,299,3.908759,driving automobile or light truck,transportation
0,050240222.jpg,607,689,599,542,605,416,691,410,704,...,-24,663,-28,623,289,555,333,3.480000,pushing car,transportation
0,033474347.jpg,876,461,815,380,779,296,742,303,706,...,-42,663,-33,674,286,631,260,2.003677,pushing car,transportation
0,082650067.jpg,1038,656,950,504,903,328,831,337,811,...,-52,729,-36,704,332,675,269,3.235820,pushing car,transportation
0,072772110.jpg,562,617,558,539,553,452,511,458,529,...,377,490,378,470,411,461,414,1.494952,pushing car,transportation
0,039361034.jpg,496,377,466,322,510,278,540,280,558,...,-3,466,-5,436,293,410,325,1.459468,pushing car,transportation
0,084761779.jpg,362,350,347,297,350,-32,317,-30,297,...,-88,288,-79,279,-57,268,-62,1.189877,pushing car,transportation


In [41]:
# Get data from 2 classes. Clean the data
# 
# Classes  : Running , Bicycling 

input_running = filter_data[filter_data["Activity"] == "running"]
input_cycling= filter_data[filter_data["Activity"] == "bicycling, mountain"]

input_cycling_filtered =  input_cycling[input_cycling[data_arr] != -1]
input_cycling_filtered =  input_cycling_filtered.dropna(axis=0)

input_running_filtered =  input_running[input_running[data_arr] != -1]
input_running_filtered =  input_running_filtered.dropna(axis=0)


In [ ]:


input = filter_data.drop(["NAME","Activity"],axis=1)
input.tail(10)

In [ ]:
#Scale the data ( Required for standard ML)


In [25]:
# Check if data is present on SPD manifold
# TODO Remove this cell
# Cannot use SPD manifold learning "5_practical_methods__simple_machine_learning_on_tangent_spaces.ipynb"

spd_input = input_running.drop(["NAME","Activity","Scale","Category"], axis=1)
spd_input_np = spd_input.to_numpy()

In [39]:
# PCA on manifolds
# 
# https://github.com/geomstats/geomstats/blob/master/notebooks/14_real_world_applications__hand_poses_analysis_in_kendall_shape_space.ipynb
#



In [40]:
# Use distance metric to perform classification on tangent space
# 
# Not present in any notebook


In [ ]:
# Perform standard ML on the dataset
#
# PCA + ML
